In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)

In [2]:
df = pd.read_csv('data/cleaned_records_bibs.csv', delimiter='|')

In [36]:
import ijson
import json
import csv
import decimal
def getAuthorsNameFormattedBySource(source, name):
    formatted_name = []
    if source == 'Google Books':
        name = name.split()
        formatted_name.append(name[-1:][0])
        formatted_name.append(', ')
        for w in name[:-1]:
             formatted_name.append(w[0] + '.')
        formatted_name = ''.join(formatted_name)
        formatted_name = formatted_name.title()
    elif source == "Scopus":
            formatted_name = name.title()
    elif source == "Web of Science":
            formatted_name = name
    return formatted_name

def getAuthorsNameBySource(source, ref):
    if source == 'Web of Science':
        return ref.split(',')[0].title()
    elif source == 'Scopus':
        return ', '.join(ref.split(',')[0:2]).title()

def parse_authors(authorsfile, edgesfile, edgesfile1):
    authorsfile = open(authorsfile, "w")
    authors_writer = csv.writer(authorsfile, delimiter='\t')
    
    edgesfile = open(edgesfile, "w")
    edge_writer = csv.writer(edgesfile, delimiter='\t')
    
    edgesfile1 = open(edgesfile1, "w")
    edge_writer1 = csv.writer(edgesfile1, delimiter='\t')
    
    user_dict = {}
    user_count= 1
    edge_dict = {}
    edge_count= 1
    edge_dict1 = {}
    edge_count1= 1
 
    edge_header = ["Source", "Target", "Label"]
    edge_writer.writerow(edge_header)
    
    edge_writer1.writerow(edge_header)

    user_header = ["Id", "name", "affiliation", "country" ,  "Label"]
    authors_writer.writerow(user_header)

    for idx, row in df.iterrows():
        authors = []
        if(type(row['author']) == float):
            continue
        names = row['author']
        if row['author'][0] == '[':
            names = eval(row['author'])
        if row['source'] in ['Scopus', 'Web of Sciences']:
            names = row['author'].split(' and ')
        for name in names:
            name = getAuthorsNameFormattedBySource(row['source'], name)
            if len(name) > 30:
                continue
            if name not in user_dict:
                user_id = "u{}".format(user_count)
                user_dict[name] = user_id, name, row['affiliation'], row['affiliation_pais']
                user_count += 1
            else:
                user_id = user_dict[name][0] 
            authors.append(user_id)
        for a in authors:
            for a1 in authors:
                if a != a1:
                    edge_dict[a + "|" + a1] = a, a1, "COLLABORATES"
                    edge_dict.pop(a1 + "|" + a, None)

        if type(row['cited-references']) != float:
            cited_authors = []
            for ref in eval(row['cited-references']):
                name  = getAuthorsNameBySource(row['source'], ref)
                if len(name) > 30:
                    continue
                if name not in user_dict:
                    user_id = "u{}".format(user_count)
                    user_dict[name] = user_id, name, "", ""
                    user_count += 1
                else:
                    user_id = user_dict[name][0] 
                cited_authors.append(user_id)
            for a in authors:
                for a1 in cited_authors:
                    if a != a1:
                        edge_dict1[a + "|" + a1] = a, a1, "CITES"
                        edge_dict1.pop(a1 + "|" + a, None)   

    for k, v in user_dict.items():
        authors_writer.writerow([v[0], v[1], v[2], v[3], "author"])  
    
    for k, v in edge_dict.items():
        edge_writer.writerow([v[0], v[1], v[2]]) 
    
    for k, v in edge_dict1.items():
        edge_writer1.writerow([v[0], v[1], v[2]]) 

    edgesfile.close()
    authorsfile.close()
    return user_count

In [37]:
user_count = parse_authors('data/authors.csv', 'data/edges_colaborates.csv','data/edges_cites.csv')

In [38]:
user_count

151080

In [27]:
df2 = pd.read_csv('data/edges_cites.csv', delimiter='\t')

In [28]:
authors = {}

In [29]:
for i, v in df2.iterrows():
    if v['Source'] not in authors.keys():
        authors[v['Source']]= 0
    if v['Target'] not in authors.keys():
        authors[v['Target']] = 0
    authors[v['Source']] = authors[v['Source']] + 1
    authors[v['Target']] = authors[v['Target']] + 1 

In [32]:
import operator
from itertools import islice
import string 
from tabulate import tabulate
import json

sorted_dict = sorted(authors.items(), key=operator.itemgetter(1), reverse=True)
ids = list(sorted_dict)

In [33]:
ids

[('u97557', 53435),
 ('u97558', 53435),
 ('u97563', 44177),
 ('u97555', 41810),
 ('u97640', 40284),
 ('u97609', 38281),
 ('u97553', 37125),
 ('u97560', 34112),
 ('u97554', 31220),
 ('u97776', 27489),
 ('u97556', 26563),
 ('u97610', 26438),
 ('u97612', 25046),
 ('u98114', 21576),
 ('u97642', 20232),
 ('u97552', 19927),
 ('u97638', 18280),
 ('u97639', 16282),
 ('u98006', 16026),
 ('u97673', 15748),
 ('u97561', 14111),
 ('u97641', 14027),
 ('u98152', 13276),
 ('u97611', 13211),
 ('u97674', 13120),
 ('u97671', 13106),
 ('u97836', 12427),
 ('u97774', 12221),
 ('u97672', 11279),
 ('u97562', 11094),
 ('u97559', 11028),
 ('u98263', 11026),
 ('u98125', 10713),
 ('u98358', 10520),
 ('u98007', 9636),
 ('u97913', 9037),
 ('u97775', 8642),
 ('u97608', 8161),
 ('u98695', 8053),
 ('u98294', 7769),
 ('u99165', 6739),
 ('u97968', 6586),
 ('u98203', 6519),
 ('u78702', 5553),
 ('u98265', 4721),
 ('u98725', 4583),
 ('u59405', 4201),
 ('u98151', 4193),
 ('u98726', 3413),
 ('u67295', 2813),
 ('u67294', 2813

In [1]:
author2 = pd.read_csv('data/authors.csv', delimiter='\t')

NameError: name 'pd' is not defined

In [44]:
def fun():
    result = []
    for w in ids:
        ida = w[0]
        for i, v in author2.iterrows():
            if w[0] == v['Id'] and len(v['name']) > 4:
                if len(result)> 20:
                    return result
                result.append((w[0], v['name'], w[1]))
                print(v['name'], w[1])

In [45]:
result = fun()

Genovese, E.D. 2813
Fox-Genovese, E. 2813
Osterhammel, J. 2190
White, E.B. 849
Stam, R. 844
Shohat, E. 844
O'Flanagan, P. 834
Winship, M.P. 816
Ingersoll, T.N. 792
Shucksmith, M. 680
Brown, D.L. 680
Shortall, S. 680
Vergunst, J. 680
Warner, M.E. 680
King, W. 636
Jackson, M. 628
Pollack, M.A. 612
Zyla, B. 600
Miller, M.J. 595
Stevens, L.M. 572
Paquette, G. 563


In [46]:
result

[('u67295', 'Genovese, E.D.', 2813),
 ('u67294', 'Fox-Genovese, E.', 2813),
 ('u21677', 'Osterhammel, J.', 2190),
 ('u33252', 'White, E.B.', 849),
 ('u81154', 'Stam, R.', 844),
 ('u81153', 'Shohat, E.', 844),
 ('u35775', "O'Flanagan, P.", 834),
 ('u57600', 'Winship, M.P.', 816),
 ('u65916', 'Ingersoll, T.N.', 792),
 ('u85577', 'Shucksmith, M.', 680),
 ('u85578', 'Brown, D.L.', 680),
 ('u85579', 'Shortall, S.', 680),
 ('u85580', 'Vergunst, J.', 680),
 ('u85581', 'Warner, M.E.', 680),
 ('u46947', 'King, W.', 636),
 ('u52014', 'Jackson, M.', 628),
 ('u633', 'Pollack, M.A.', 612),
 ('u75239', 'Zyla, B.', 600),
 ('u10972', 'Miller, M.J.', 595),
 ('u64285', 'Stevens, L.M.', 572),
 ('u45715', 'Paquette, G.', 563)]